In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import json
import os

import anndata as ad
import numpy as np
import pandas as pd
import sklearn
import torch
import tqdm

import celltrip

os.environ['AWS_PROFILE'] = 'waisman-admin'


# Load Data and Policy

In [ ]:
# Read data files
# adata_prefix = 's3://nkalafut-celltrip/Dyngen'
adata_prefix = '../data/Dyngen'
adatas = celltrip.utility.processing.read_adatas(
    's3://nkalafut-celltrip/dyngen/logcounts.h5ad',
    's3://nkalafut-celltrip/dyngen/counts_protein.h5ad',
    backed=True)
# Model location and name (should be prefix for .weights, .pre, and .mask file)
prefix, training_step = 's3://nkalafut-celltrip/checkpoints/Dyngen-250920', 800
# prefix, training_step = 's3://nkalafut-celltrip/checkpoints/Dyngen-251015', 800
# Generate or load preprocessing
preprocessing = celltrip.utility.processing.Preprocessing().load(f'{prefix}.pre')
with celltrip.utility.general.open_s3_or_local(f'{prefix}.mask', 'rb') as f:
    mask = np.loadtxt(f).astype(bool)
adatas[0].obs['Training'] = mask
# Create sample env (kind of a dumb workaround, TODO)
m1, m2 = [preprocessing.transform(ad[:2].X, subset_modality=i)[0] for i, ad in enumerate(adatas)]
env = celltrip.environment.EnvironmentBase(
    torch.tensor(m1), torch.tensor(m2), compute_rewards=False, dim=8).eval().to('cuda')
# Load policy
policy = celltrip.policy.create_agent_from_env(
    env, forward_batch_size=1_000, vision_size=1_000).eval().to('cuda')
policy.load_checkpoint(f'{prefix}-{training_step:04}.weights');


# Perform Significance Estimation

In [4]:
# # Params
# np.random.seed(42)
# genes_to_survey = adatas[0].var_names
# sim_time = 5.

# # Mute warnings (array wrap and indexing)
# import warnings
# warnings.simplefilter('ignore')

# # Create anndata
# ad_pert = ad.AnnData(obs=adatas[0].obs, var=pd.DataFrame(index=[f'Feature {i}' for i in range(env.dim)]))
# # ad_pert0 = ad.AnnData(obs=adatas[0].obs, var=adatas[0].var)
# # ad_pert1 = ad.AnnData(obs=adatas[1].obs, var=adatas[1].var)
# def add_layers(states, gene):
#     ad_pert.layers[gene] = states
#     # ad_pert0.layers[gene] = states_0
#     # ad_pert1.layers[gene] = states_1

# # Add results
# results = []
# def add_record(states, states_0, states_1, gene, ct):
#     results.append({
#         'Gene': gene, 'Cell Type': ct,
#         'Effect Size (Latent)': np.square(states[-1] - states[0]).mean(),
#         'Trajectory Length (Latent)': np.square(states[1:] - states[:-1]).mean(axis=(-2, -1)).sum(),
#         'Effect Size (Modality 0)': np.square(states_0[-1] - states_0[0]).mean(),
#         'Trajectory Length (Modality 0)': np.square(states_0[1:] - states_0[:-1]).mean(axis=(-2, -1)).sum(),
#         'Effect Size (Modality 1)': np.square(states_1[-1] - states_1[0]).mean(),
#         'Trajectory Length (Modality 1)': np.square(states_1[1:] - states_1[:-1]).mean(axis=(-2, -1)).sum()})
    
# # Reset function
# def reset_env(env, steady_pos, steady_vel, modal_dict={}):
#     env.set_max_time(sim_time).reset()  # TODO: Maybe longer?, early stopping?
#     env.set_positions(steady_pos)
#     env.set_velocities(steady_vel)  # Maybe 0 manually?
#     for k, v in modal_dict.items():
#         env.modalities[k] = v

# # Running function
# def run_and_record(samples, env, policy, preprocessing, gene, gene_idx):
#     # Run and impute
#     states = celltrip.train.simulate_until_completion(
#         env, policy,
#         env_hooks=[
#             celltrip.utility.hooks.clamp_inverted_features_hook(
#                 gene_idx, preprocessing, feature_targets=0., modality_idx=0),
#         ],
#         action_hooks=[
#             celltrip.utility.hooks.move_toward_targets_hook(
#                 gene_idx, feature_targets=0., pinning=policy.pinning[0],
#                 preprocessing=preprocessing, modality_idx=0,
#                 factor=1, device=env.device),
#         ],
#         store_states='cpu')[-1]
#     states_pos = states[..., :env.dim]
#     with torch.no_grad():
#         imputed_states_0 = policy.pinning[0](states_pos.to('cuda')).detach().cpu().numpy()
#         imputed_states_1 = policy.pinning[1](states_pos.to('cuda')).detach().cpu().numpy()
#     imputed_states_0, = preprocessing.inverse_transform(imputed_states_0, subset_modality=0)
#     imputed_states_1, = preprocessing.inverse_transform(imputed_states_1, subset_modality=1)
#     # Record
#     add_layers(states_pos.numpy()[-1], gene)  # , imputed_states_0[-1], imputed_states_1[-1]
#     add_record(states_pos.numpy(), imputed_states_0, imputed_states_1, gene, 'All')
#     for ct in adatas[0][samples].obs['traj_sim'].unique():
#         add_record(
#             states_pos[:, adatas[0][samples].obs['traj_sim']==ct].numpy(),
#             imputed_states_0[:, adatas[0][samples].obs['traj_sim']==ct],
#             imputed_states_1[:, adatas[0][samples].obs['traj_sim']==ct],
#             gene, ct)

# # Subset and preprocess the data
# samples = adatas[0].obs.index
# raw_m1 = celltrip.utility.processing.chunk_X(adatas[0][samples], chunk_size=2_000)
# m1, m2 = [
#     celltrip.utility.processing.chunk_X(
#         ad[samples], chunk_size=2_000,
#         func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
#         for i, ad in enumerate(adatas)]

# # Initialize environment
# env = celltrip.environment.EnvironmentBase(
#     torch.tensor(m1), torch.tensor(m2), compute_rewards=False, dim=8).eval(time_scale=1).to('cuda')

# # Simulate to steady state
# env.reset()
# celltrip.train.simulate_until_completion(env, policy)
# steady_pos, steady_vel = (env.pos, env.vel)

# # Run control
# reset_env(env, steady_pos, steady_vel)
# add_layers(steady_pos.cpu().numpy(), 'Steady')
# run_and_record(samples, env, policy, preprocessing, 'Control')

# # Perturb
# for gene in tqdm.tqdm(genes_to_survey, miniters=10, maxinterval=30):
#     # Get gene idx and run
#     gene_idx = np.argwhere(adatas[0].var_names==gene).flatten()
#     reset_env(env, steady_pos, steady_vel)  # {0: torch.tensor(m1).cuda()}
#     run_and_record(samples, env, policy, preprocessing, gene, gene_idx)

# # Convert and save
# pd.DataFrame(results).to_csv('../plots/dyngen/knockdown.csv', index=None)
# ad_pert.write_h5ad('../plots/dyngen/knockdown_results.h5ad')
# # ad_pert0.write_h5ad('../plots/dyngen/knockdown_results_modality_0.csv')
# # ad_pert1.write_h5ad('../plots/dyngen/knockdown_results_modality_1.csv')


# Perform Module Knockdown

In [5]:
# Load modules
with celltrip.utility.general.open_s3_or_local('../data/dyngen/dyngen_tfs.json', 'rb') as f:
    tf_modules = json.load(f)
    tf_modules.pop('NaN')

# Params
np.random.seed(42)
sim_time = 128.

# Mute warnings (array wrap and indexing)
import warnings
warnings.simplefilter('ignore')

# Create anndata
ad_pert = ad.AnnData(obs=adatas[0].obs, var=pd.DataFrame(index=[f'Feature {i}' for i in range(env.dim)]))
# ad_pert0 = ad.AnnData(obs=adatas[0].obs, var=adatas[0].var)
# ad_pert1 = ad.AnnData(obs=adatas[1].obs, var=adatas[1].var)
def add_layers(states, gene):
    ad_pert.layers[gene] = states
    # ad_pert0.layers[gene] = states_0
    # ad_pert1.layers[gene] = states_1

# Add results
results = []
def add_record(states, states_0, states_1, gene, ct):
    results.append({
        'Gene': gene, 'Cell Type': ct,
        'Effect Size (Latent)': np.square(states[-1] - states[0]).mean(),
        'Trajectory Length (Latent)': np.square(states[1:] - states[:-1]).mean(axis=(-2, -1)).sum(),
        'Effect Size (Modality 0)': np.square(states_0[-1] - states_0[0]).mean(),
        'Trajectory Length (Modality 0)': np.square(states_0[1:] - states_0[:-1]).mean(axis=(-2, -1)).sum(),
        'Effect Size (Modality 1)': np.square(states_1[-1] - states_1[0]).mean(),
        'Trajectory Length (Modality 1)': np.square(states_1[1:] - states_1[:-1]).mean(axis=(-2, -1)).sum()})
    
# Reset function
def reset_env(env, steady_pos, steady_vel, modal_dict={}):
    env.set_max_time(sim_time).reset()  # TODO: Maybe longer?, early stopping?
    env.set_positions(steady_pos)
    env.set_velocities(steady_vel)  # Maybe 0 manually?
    for k, v in modal_dict.items():
        env.modalities[k] = v

# Running function
def run_and_record(samples, env, policy, preprocessing, gene, gene_idx):
    # Run and impute
    states = celltrip.train.simulate_until_completion(
        env, policy,
        env_hooks=[
            celltrip.utility.hooks.clamp_inverted_features_hook(
                gene_idx, preprocessing, feature_targets=0., modality_idx=0),
        ],
        action_hooks=[
            celltrip.utility.hooks.move_toward_targets_hook(
                gene_idx, feature_targets=0., pinning=policy.pinning[0],
                preprocessing=preprocessing, modality_idx=0,
                factor=1, device=env.device),
        ],
        skip_states=100, store_states='cpu')[-1]
    states_pos = states[..., :env.dim]
    with torch.no_grad():
        imputed_states_0 = policy.pinning[0](states_pos.to('cuda')).detach().cpu().numpy()
        imputed_states_1 = policy.pinning[1](states_pos.to('cuda')).detach().cpu().numpy()
    imputed_states_0, = preprocessing.inverse_transform(imputed_states_0, subset_modality=0)
    imputed_states_1, = preprocessing.inverse_transform(imputed_states_1, subset_modality=1)
    # Record
    add_layers(states_pos.numpy()[-1], gene)  # , imputed_states_0[-1], imputed_states_1[-1]
    add_record(states_pos.numpy(), imputed_states_0, imputed_states_1, gene, 'All')
    for ct in adatas[0][samples].obs['traj_sim'].unique():
        add_record(
            states_pos[:, adatas[0][samples].obs['traj_sim']==ct].numpy(),
            imputed_states_0[:, adatas[0][samples].obs['traj_sim']==ct],
            imputed_states_1[:, adatas[0][samples].obs['traj_sim']==ct],
            gene, ct)

# Subset and preprocess the data
samples = adatas[0].obs.index
raw_m1 = celltrip.utility.processing.chunk_X(adatas[0][samples], chunk_size=2_000)
m1, m2 = [
    celltrip.utility.processing.chunk_X(
        ad[samples], chunk_size=2_000,
        func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
        for i, ad in enumerate(adatas)]

# Initialize environment
env = celltrip.environment.EnvironmentBase(
    torch.tensor(m1), torch.tensor(m2), compute_rewards=False, dim=8).eval(time_scale=1).to('cuda')

# Simulate to steady state
env.reset()
celltrip.train.simulate_until_completion(env, policy)
steady_pos, steady_vel = (env.pos, env.vel)

# Run control
reset_env(env, steady_pos, steady_vel)
add_layers(steady_pos.cpu().numpy(), 'Steady')
run_and_record(samples, env, policy, preprocessing, 'Control', [])

# Perturb
for module, genes in (pbar := tqdm.tqdm(tf_modules.items())):
    # Get gene idx and run
    pbar.set_description(module)
    gene_idx = np.argwhere(np.isin(adatas[0].var_names, genes)).flatten()
    reset_env(env, steady_pos, steady_vel)  # {0: torch.tensor(m1).cuda()}
    run_and_record(samples, env, policy, preprocessing, module, gene_idx)

# Convert and save
pd.DataFrame(results).to_csv('../plots/dyngen/knockdown_full.csv', index=None)
ad_pert.write_h5ad('../plots/dyngen/knockdown_full_results.h5ad')



  0%|                                                                                                                                                                                      | 0/62 [00:00<?, ?it/s]


Burn1:   0%|                                                                                                                                                                               | 0/62 [00:00<?, ?it/s]


Burn1:   2%|██▋                                                                                                                                                                    | 1/62 [00:49<50:16, 49.44s/it]


Burn2:   2%|██▋                                                                                                                                                                    | 1/62 [00:49<50:16, 49.44s/it]


Burn2:   3%|█████▍                                                                                                                                                                 | 2/62 [01:10<32:30, 32.51s/it]

...


G5:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 60/62 [26:47<01:06, 33.08s/it]


G5:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 61/62 [27:20<00:32, 32.83s/it]


G6:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 61/62 [27:20<00:32, 32.83s/it]


G6: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [27:51<00:00, 32.46s/it]


G6: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [27:51<00:00, 26.96s/it]

In [ ]:
# ad_pert = ad.read_h5ad('../plots/dyngen/knockdown_full_results.h5ad')
# control_gex, = preprocessing.inverse_transform(policy.pinning[0](torch.tensor(ad_pert.layers['Control']).cuda()).detach().cpu().numpy(), subset_modality=0)
# pert_gex, = preprocessing.inverse_transform(policy.pinning[0](torch.tensor(ad_pert.layers['D2']).cuda()).detach().cpu().numpy(), subset_modality=0)
# prioritized_genes = adatas[0].var.index[np.abs(pert_gex - control_gex).mean(axis=0).argsort()[::-1]].to_numpy()